In [1]:
### Load Documents
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

## Load Text
loader = TextLoader("txt/mgs5_cassette_tapes.txt")
documents = loader.load()

## Split Text
# Initialize the RecursiveCharacterTextSplitter
# chunk_size: The maximum size of each chunk (in characters by default).
# chunk_overlap: The number of characters to overlap between consecutive chunks,
#                helping to maintain context.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,  # Use character length for chunk size
    is_separator_regex=False, # Treat separators literally
)

# Split the loaded documents
split_docs = text_splitter.split_documents(documents)

# Print the resulting chunks
print(f"Number of original documents: {len(documents)}")
print(f"Number of split chunks: {len(split_docs)}\n")

#for i, chunk in enumerate(split_docs):
#    print(f"Chunk {i+1}:\n{chunk.page_content}\n---")
print(f"split_docs[0].page_content: {split_docs[0].page_content}\n")
print(f"split_docs[0].metadata: {split_docs[0].metadata}\n")


Number of original documents: 1
Number of split chunks: 4920

split_docs[0].page_content: In Metal Gear Solid V: The Phantom Pain, the player can access cassette tapes, either at hand or by

split_docs[0].metadata: {'source': 'txt/mgs5_cassette_tapes.txt'}



In [2]:
### Connect to OCI embeddings and generative AI
from langchain_community.embeddings import OCIGenAIEmbeddings
from langchain_community.chat_models import ChatOCIGenAI

oci_embeddings = OCIGenAIEmbeddings(
    model_id="cohere.embed-english-light-v3.0",
    service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
    compartment_id="ocid1.compartment.oc1..aaaaaaaa52sp42nqmtwwzzvmp5mmldri26razhrbyw7cvixmims7p5crsg7a",
)

oci_chat = ChatOCIGenAI(
    model_id="cohere.command-a-03-2025",
    service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
    compartment_id="ocid1.compartment.oc1..aaaaaaaa52sp42nqmtwwzzvmp5mmldri26razhrbyw7cvixmims7p5crsg7a",
    model_kwargs={"temperature": 0.7, "max_tokens": 500},
)

print("Connected to OCI embeddings and generative AI.")

Connected to OCI embeddings and generative AI.


In [ ]:
### Embed Documents
#l = len(split_docs)
#embeddings = []
#for i in range(l // 16 + 1):
#    #print(f"Embedding from index {i*16} to {(i + 1) * 16}...")
#    subdocs = [item.page_content for item in split_docs[i * 16: (i + 1) * 16]]
#    embeddings.extend(oci_embeddings.embed_documents(subdocs))
#
#print("Documents embedded.")

In [ ]:
#### Build Vector Store (FAISS)
#from langchain_community.vectorstores import FAISS
#
#texts = [item.page_content for item in split_docs]
#text_embedding_pairs = [(text, embed) for text, embed in zip(texts, embeddings)]
#vectorstore = FAISS.from_embeddings(text_embedding_pairs, oci_embeddings)
#
#print("FAISS vector store built.")

In [ ]:
#from langchain_community.vectorstores import Chroma
#from langchain_core.documents import Document
#
## 2. Define some documents to add to the vector store
#documents = [
#    Document(page_content="The quick brown fox jumps over the lazy dog.", metadata={"source": "fable"}),
#    Document(page_content="Artificial intelligence is transforming industries worldwide.", metadata={"source": "technology"}),
#    Document(page_content="The capital of France is Paris.", metadata={"source": "geography"}),
#]
#
## 3. Create a Chroma vector store instance
## You can specify a persist_directory to save the data to disk
#persist_directory = "./chroma_db"
#vectorstore = Chroma.from_documents(
#    documents=documents,
#    embedding=embeddings,
#    persist_directory=persist_directory,
#    collection_name="my_documents"
#)
#print("Chroma vector store built.")

In [ ]:
##### Build Vector Store (In-Memory)
#from langchain_core.vectorstores import InMemoryVectorStore
#
## Create an in-memory vector store
#vectorstore = InMemoryVectorStore(oci_embeddings)
#vectorstore.add_documents(documents=split_docs)
#
#print("In-memory vector store built.")

In [3]:
### Build Vector Store (FAISS)
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(split_docs, oci_embeddings)

print("FAISS vector store built.")

FAISS vector store built.


In [11]:
### Build Chain (OCI chat-based Retrieval QA)
from langchain_core.prompts import PromptTemplate
#from langchain_classic.chains import RetrievalQA
from langchain.chains import RetrievalQA

retriever = vectorstore.as_retriever(retriever_kwargs={"similarity_top_k": 3})

#rag_prompt_template = """Answer the question based only on the following context:
#{context}
#Question: {question}
#"""

#rag_prompt_template = """Try to the question using only the following context, but, if that fails, use your general knowledge.  You don't have to mention whether you did or did not use the context.
#{context}
#Question: {question}
#"""

rag_prompt_template = """Try to the question using only the following context, but, if that fails, use your general knowledge.
{context}
Question: {question}
"""

rag_prompt = PromptTemplate.from_template(rag_prompt_template)

rag = RetrievalQA.from_chain_type(
    llm=oci_chat,
    retriever=retriever,
    chain_type_kwargs={"prompt": rag_prompt,},
)

#rag = RetrievalQA.from_chain_type(
#    llm=oci_chat,
#    chain_type="stuff",
#    retriever=retriever,
#    return_source_documents=True,
#    verbose=False,
#)

print("RAG chain built.")

RAG chain built.


In [12]:
### Invoke the chain (unit test)
print(rag.invoke("Who is Kazuhira?"))
print(rag.invoke("Are Venom Snake and Revolver Ocelot friends?"))
print(rag.invoke("What is the name of Anderson's AI?"))
print(rag.invoke("What is Zero's fear once the Cold War is over?"))
print(rag.invoke("How many cassette tapes are there in Metal Gear Solid 5?"))

{'query': 'Who is Kazuhira?', 'result': 'Based on the provided context, Kazuhira is Benedict Miller, referred to as Benedict "Kazuhira" Miller. The context mentions his research and epiphany related to "The Hamburgers of Kazuhira Miller," but it does not provide further details about his identity beyond his name. \n\nSince the context does not offer additional information, I cannot provide more details using only the given text. However, using general knowledge (as the context fails to provide a complete answer), Kazuhira Miller is a character from the *Metal Gear* video game series, specifically from *Metal Gear Solid V: The Phantom Pain*. He is a mentor figure to the protagonist, Venom Snake, and plays a significant role in the game\'s story.'}
{'query': 'Are Venom Snake and Revolver Ocelot friends?', 'result': "Based on the provided context, it is not explicitly stated whether Venom Snake and Revolver Ocelot are friends. The dialogue suggests a level of familiarity and a working rel

In [13]:
### Use ADS to deploy the unit-tested chain
## Create the ADS deployment object
import tempfile
from ads.llm.deploy import ChainDeployment

artifact_dir = tempfile.mkdtemp()

ads_deployment = ChainDeployment(
    chain=rag,
    artifact_dir=artifact_dir,
    force_overwrite=True
)

ads_deployment.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Available     Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Not Available Local tested .predict from score.py                               
save()    Not Available Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  UNKNOWN       Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [7]:
# Prepare the ADS deployment
ads_deployment.prepare(
    inference_conda_env="automlx251_p311_cpu_x86_64_v2",
    inference_python_version="3.11",
    force_overwrite=True,
)

# Summarize the checkpoitn ADS workflow status
ads_deployment.summary_status()

  warnings.warn("slug will be deprecated. Provide conda pack path instead.")

ERROR:root:Error occurred in attempt to extract the list of the service conda environments from the object storage for bucket 'service-conda-packs' and namespace 'id19sfcrra6z'. Please make sure that you've provided correct bucket and namespace.
INFO:ADS:To auto-extract taxonomy metadata the model must be provided. Supported models: keras, lightgbm, pytorch, sklearn, tensorflow, pyspark, and xgboost.


Actions Needed
Step      Status        Details                                                                                                                                                         
initiate  Done          Initiated the model                                                                                                                                             
prepare() Done          Generated runtime.yaml                                                                                                                                          
                        Generated score.py                                                                                                                                              
          Needs Action  Serialized model                                     Model is not automatically serialized. Serialize the model as `model.pkl` and save to the /tmp/tmp3aojz77n.
          Done          Populated metadata(Custom, Taxonomy and Provenance)                                                                                                             
verify()  Available     Local tested .predict from score.py                                                                                                                             
save()    Available     Conducted Introspect Test                                                                                                                                       
                        Uploaded artifact to model catalog                                                                                                                              
deploy()  UNKNOWN       Deployed the model                                                                                                                                              
predict() Not Available Called deployment predict endpoint

In [14]:
# Save the ADS model
ads_deployment.verify("Who wants to cook hamburgers once the conflict is over?")

# Summarize the checkpoitn ADS workflow status
ads_deployment.summary_status()

ERROR - Exception
Traceback (most recent call last):
  File "/home/datascience/conda/automlx251_p311_cpu_x86_64_v2/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_16147/267121191.py", line 2, in <module>
    ads_deployment.verify("Who wants to cook hamburgers once the conflict is over?")
  File "/home/datascience/conda/automlx251_p311_cpu_x86_64_v2/lib/python3.11/site-packages/ads/model/generic_model.py", line 1277, in verify
    raise ArtifactsNotAvailableError
ads.model.generic_model.ArtifactsNotAvailableError: Model artifacts are either not generated or not available locally.
ArtifactsNotAvailableError: Model artifacts are either not generated or not available locally.

In [15]:
# Save the ADS model
model_id = ads_deployment.save()

# Summarize the checkpoitn ADS workflow status
ads_deployment.summary_status()

ERROR - Exception
Traceback (most recent call last):
  File "/home/datascience/conda/automlx251_p311_cpu_x86_64_v2/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_16147/853832723.py", line 2, in <module>
    model_id = ads_deployment.save()
               ^^^^^^^^^^^^^^^^^^^^^
  File "/home/datascience/conda/automlx251_p311_cpu_x86_64_v2/lib/python3.11/site-packages/ads/model/generic_model.py", line 2134, in save
    raise ArtifactsNotAvailableError
ads.model.generic_model.ArtifactsNotAvailableError: Model artifacts are either not generated or not available locally.
ArtifactsNotAvailableError: Model artifacts are either not generated or not available locally.